# Курс "Программирование на языке Python. Уровень 4. Анализ и визуализация данных на языке Python. Библиотеки numpy, pandas, matplotlib"

## Модуль 9. Временные ряды в pandas

1. Работа с датами в Python
2. Объект TimeSeries в Pandas - особенности, срезы и пр.
4. Частоты и периоды, ресемплинг, сдвиг
5. Статистический анализ временных рядов: "скользящее окно" и STL-декомпозиция

Для работы загрузите в директорию ```data``` следующие данные (их можно загрузить прямо отсюда, через "Сохранить как..."):
- [data/macrodata.csv](https://github.com/easyise/spec_python_courses/raw/master/python04-analysis/data/macrodata.csv)
- [data/web_traffic.tsv](https://github.com/easyise/spec_python_courses/raw/master/python04-analysis/data/web_traffic.tsv)
- [data/monthly-temperature-in-celsius-j.csv](https://github.com/easyise/spec_python_courses/raw/master/python04-analysis/data/monthly-temperature-in-celsius-j.csv)
- [data/monthly-australian-wine-sales.csv](https://github.com/easyise/spec_python_courses/raw/master/python04-analysis/data/monthly-australian-wine-sales.csv)


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from datetime import datetime
from datetime import date
from datetime import timedelta

import statsmodels.api as sm

plt.rcParams['figure.figsize'] = (10.0, 10.0)
%matplotlib inline


### Дата и время в Python

Модуль для работы с датами - ```datetime```.

В нем следующие классы:
- datetime.date
- datetime.timedelta
- datetime.datetime

При работе ориентируемся на ISO-стандарт хранения даты и времени в виде текстовой строки: \
```YYYY-MM-DD HH:MM:SS.mmm```.

Этот старндарт используется в SQL, JavaScript и других языках программирования, принимается множеством API.

Для создания даты из такой строки используйте метод ```datetime.fromisoformat()```.
Сохранить дату в ISO-формате: ```datetime.isoformat()```.

Текущее местное время: ```datetime.today()```. Время по UTC: ```datetime.utcnow()```

**ВНИМАНИЕ!** Смещение часового пояса относительно UTC используйте только если вы действительно работаете с данными в разных часовых поясах. Если это не требуется (и не может портребоваться в перспективе) - не используйте этот параметр.

In [ ]:
now = datetime.today()
print(now.timestamp())
print(now.isoformat())

unixEpoch = datetime.fromisoformat("1970-01-01 03:00")
print(unixEpoch.isoformat())

# то же самое - с датами
today = date.today()
print(today.isoformat())

# Можно создать дату, зная месяц, год и число:
gagarin_date = date(1961, 4, 12)
print(gagarin_date)

Преобразование дат в строку: ```datetime.strftime()```

Пример преобразования в строку в соответствии с ISO-форматом:
```somedate.strftime('%Y-%m-%d %H:%M:%S')```

#### ЗАДАНИЕ. Преобразуйте сегодняшнюю дату в формат ДД.ММ.ГГГГ ЧЧ:ММ

In [ ]:
# ваш код здесь


#### Разница во времени, временные интервалы

Используем класс ```timedelta```. "Дельты" можно складывать с датами и датой/временем, друг с другом, делить, умножать и так далее.

In [ ]:
delta = timedelta(
    days=50,\
    seconds=27,\
    microseconds=10,\
    milliseconds=29000,\
    minutes=5,\
    hours=8,\
    weeks=2\
)

now_plus_delta = now + delta
print(now_plus_delta.isoformat())

mins_15 = timedelta(minutes = 15) 

now_plus_half_hour = datetime.today() + mins_15*2
print(now_plus_half_hour)

print(now_plus_delta > now_plus_half_hour)

#### ПРАКТИКА

1. Создайте массив numpy из 10 дат, которые соответствуют текущей и далее + 10 дней.

2. Создайте объект series, который включает все значаения времени, которые отстают от текущего на час, полчаса, 15 минут и так далее с точностью до минуты.

In [ ]:
today = datetime.today()
# ваш код здесь


### Объект TimeSeries

Создадим простой временной ряд в pandas:

In [ ]:
n = 10
values = np.random.randn(n)
dates = [ datetime.fromisoformat('2011-10-19') + i*timedelta(days=2) for i in range(n) ]
ts = pd.Series(values, index=dates)
ts

In [ ]:
ts.index

In [ ]:
ts.index.dtype # данные в индексе хранятся с точностью до наносекунды

К временному ряду можно обращаться по порядковому номеру позиции или по дате в iso-формате:

In [ ]:
ts[0]

In [ ]:
ts['2011-10-21']

...а также по части даты, которая может быть интерпретирована как день, месяц, год (час, минута, секунда и пр.)

In [ ]:
n = 1000
ts_long = pd.Series(np.random.randn(n), \
               index=[ datetime.fromisoformat('2020-10-19') + i*timedelta(days=2) for i in range(n)])

In [ ]:
ts_long['2021']

In [ ]:
ts_long['2020-12']

...срезы тоже работают:

In [ ]:
ts_long['2020-10-21':'2020-11-02']

In [ ]:
ts_long[:datetime.today()]

Также для этого можно использовать метод ```truncate()``` - это выражение читается как "обрежь все до ...":

In [ ]:
ts_long.truncate(before='2026-03-01')

__ЗАДАНИЕ:__ выведите данные, которые у нас есть в ряду ts_long на апрель 2023 года, просуммируйте их

In [ ]:
# ваш код здесь


### Диапазоны дат, ресемплинг, сдвиг

Для генерации диапазонов дат можно использовать метод ```date_range()```:

In [ ]:
index = pd.date_range('2020-10-01', '2020-12-01')
index

Можно задать не диапазон дат, а стартовое или конечное значение и количество элементов:

In [ ]:
pd.date_range('2020-10-19 15:00', periods=20)

In [ ]:
pd.date_range(end='2020-11-02', periods=20)

Обратите внимание на свойство ```freq``` - параметр с таким именем задаст частоту генерации временного ряда. Наиболее часто используемые значения этого параметра следующие:
- B - каждый рабочий день
- D - каждый календарный день
- W - каждая неделя
- MS - каждый первый день месяца
- M - каждый последний день месяца
- QS- начало квартала
- Q - конец квартала
- AS, YS - начало года
- A, Y- конец года
- H - каждый час
- T, min - каждая минута
- S - каждая секундна

... и так далее, до наносекунд.

Можно использовать более сложные сочетания значений. Например, для анализа финансового года/квартала в случае, когда год заканчивается в апреле:
 - Q-APR - ежеквартально, первый квартал будет заканчиваться в апреле.
 - A-APR - ежегодно с окончанием периода в апреле.
 
Пример:

In [ ]:
ts = pd.date_range('2020-01-01', periods=4, freq='QS-APR')
ts

In [ ]:
ts_A = pd.date_range('2020-01-01', periods=4, freq='A-APR')
ts_A

Также можно задавать кратность частоты:

In [ ]:
pd.date_range(datetime.today(), periods=10, freq='2h30min')

Еще часто используются периоды с обращением в определенный день недели месяца, например вот код для генерации последовательности для каждой третьей пятницы месяца:

In [ ]:
pd.date_range(date.today(), periods=10, freq='WOM-3FRI')

Диапазон данных можно сдвигать с опережением и запаздыванием с помощью метода ```shift()```. Это делается, например, для сравнения относительных изменений в одном или нескольких временных рядах.

In [ ]:
ts = pd.Series(np.random.randn(4),
               index=pd.date_range('2000-01-01', periods=4, freq='M'))
ts

In [ ]:
ts.shift(2) # сдвиг на 2 периода вперед

In [ ]:
ts.shift(-2) # сдвиг на 2 периода назад

НО! Такой "фокус" не пройдет, если у вас индекс состоит не из ```PeriodObject```, а временных меток (```TimeStamp```).

In [ ]:
rng = pd.date_range('2000-01-01', periods=3, freq='M')
ts = pd.Series(np.random.randn(3), index=rng)
ts.index

Преобразовать индекс из временных меток в периоды можно функцией ```to_period()```.

In [ ]:
pts = ts.to_period()
pts.index

In [ ]:
rng = pd.date_range('1/29/2000', periods=6, freq='D')
ts2 = pd.Series(np.random.randn(6), index=rng)
ts2


In [ ]:
pts2 = ts2.to_period('M')
pts2

Обратное преобразование выполняется методом ```to_timestamp()```

In [ ]:
pts2.to_timestamp(how='end')

__ЗАДАНИЕ__: Загрузите данные с ежечасной статистикой веб-трафика ```data/web_traffic.tsv```. Увы, дата начала сбора статистики неизвестна, известно, что последний день - 10 ноября 2019 года. Задайте индекс в виде даты/времени.

In [ ]:
web_traffic = pd.read_csv('data/web_traffic.tsv', sep='\t', header=None, names=['Day', 'ReqsPerHour'])
web_traffic.head()

# ваш код здесь

Как "вытащить" периоды из загруженного датасета:

In [ ]:
data = pd.read_csv('data/macrodata.csv')
data.head(5)

In [ ]:
data.year
data.quarter

In [ ]:
index = pd.PeriodIndex(year=data.year, quarter=data.quarter,
                       freq='Q-DEC')
index

In [ ]:
data.index = index
data.infl.plot()

__ЗАДАНИЕ__:

Загрузите датасет c данными по месячным температурам, постройте график. Сдвиньте график на год, на 2 года и на 5 лет, сравните. 

In [ ]:
plt.figure(figsize=(10,6))
temps = pd.read_csv('data/monthly-temperature-in-celsius-j.csv',
                      index_col=['Month'], parse_dates=['Month'], 
                      dayfirst=True)
# ваш код здесь


### Ресемплинг


Повышающая дискретизация, upsampling

In [ ]:
rng = pd.date_range('2020-01-01', periods=12, freq='T')
ts = pd.Series(np.arange(12), index=rng)
ts

In [ ]:
ts.resample('5min', closed='right').sum()

Понижающая дискретизация, downsampling

In [ ]:
frame = pd.DataFrame(np.random.randn(2, 4),
                     index=pd.date_range('1/1/2000', periods=2,
                                         freq='W-WED'),
                     columns=['Colorado', 'Texas', 'New York', 'Ohio'])
frame

In [ ]:
df_daily = frame.resample('D').asfreq()
df_daily

In [ ]:
frame.resample('D').ffill()

In [ ]:
frame.resample('D').ffill().plot()

__ЗАДАНИЕ__: Для температурного датасета проеобразуйте период в "весна"-"лето"-"осень"-"зима" и постройте график. Сделайте сдвиги.

In [ ]:
temperatures = pd.read_csv('data/monthly-temperature-in-celsius-j.csv',
                      index_col=['Month'], parse_dates=['Month'], 
                      dayfirst=True)
# ваш код здесь

### Статистический анализ временных рядов

Пример "скользящего среднего".

In [ ]:
close_px_all = pd.read_csv('data/stock_px_2.csv', \
                           parse_dates=True, index_col=0)
close_px = close_px_all[['AAPL', 'MSFT', 'XOM']]
close_px = close_px.resample('B').ffill()
plt.rcParams['figure.figsize'] = (10.0, 10.0)
close_px['AAPL'].plot()

In [ ]:
close_px = close_px['2008':'2008']

close_px.AAPL.plot()
#close_px.AAPL.rolling(50).mean().plot()
close_px.AAPL.rolling(60).mean().plot()
#close_px.AAPL.rolling(40).mean().plot()

#### STL-декомпозиция ряда

Для анализа ряда с ярко выраженной сезонностью используется STL-декомпозиция.

Она работает для двух моделей:

1) Аддитивная модель: используется, когда отколнения от тренда не сильно варьируется в определенных временных интервалах. Тогда наш показатель может быть представлен суммой, вычисляемой по формуле:\
        y(t) = уровень(t) + тренд(t) + сезонность(t) + шум(t)
    
2) Мультипликативная модель - используется, когда размах сезонности имеет выраженную зависимость от времени и тренда:\
        y(t) = уровень(t) * тренд(t) * сеознность(t) * шум(t)
        
Рассмотрим на примере датасета "продажи австралийского вина".

In [ ]:
plt.figure(figsize=(16,4))
wine = pd.read_csv('data/monthly-australian-wine-sales.csv',
                   index_col=['month'], parse_dates=['month'], 
                   dayfirst=True)
wine.sales = wine.sales * 1000

wine.sales.plot()
plt.ylabel('Wine sales')

In [ ]:
sm.tsa.seasonal_decompose?

In [ ]:
decomposition = sm.tsa.seasonal_decompose(wine.sales)
fig = decomposition.plot()
fig.set_figwidth(16)
fig.set_figheight(16)
plt.show()

Выведем отдельно тренд:

In [ ]:
plt.figure(figsize=(16,4))
plt.plot(decomposition.trend.index, decomposition.trend, c='red')

__ЗАДАНИЕ__: выполните STL-декомпозицию для статистики по запросам веб-сервера - за последние 3 дня, последнюю неделю и предшествующую ей неделю.

In [ ]:
web_traffic = pd.read_csv('data/web_traffic.tsv', sep='\t', header=None, names=['Day', 'ReqsPerHour'], index_col="Day")
web_traffic.head()
# ваш код здесь
